# Data Science Part Time Course

## Week 5 - Lesson 2 - Lab 2: Databases with Python

In this Lab the goal is connect to a database (created form a local file), read the data into Python and interact with it.

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [ ]:
import sqlite3
conn = sqlite3.connect('enron.db') 
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

Query the `sqlite_master` table to retrieve the **schema** of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [ ]:
fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='MessageBase';").fetchall()
print (''.join(fields[0]))

fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='RecipientBase';").fetchall()
print (''.join(fields[0]))

fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='EmployeeBase';").fetchall()
print (''.join(fields[0]))

1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [ ]:
results = c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
for row in results:
    print (row)

In [ ]:
results = c.execute("SELECT * FROM MessageBase LIMIT 5;").fetchall()
for row in results:
    print (row)

In [ ]:
results = c.execute("SELECT * FROM RecipientBase LIMIT 10;").fetchall()
for row in results:
    print (row)
    
# The first field is message id, the second is recipient number, and the third is the id of the recipient.
# mid, rno, to_eid

Now try other SQL statements on the local database, such as SELECT .... FROM ... WHERE ....

For example, print the records of EmployeeBase where the Gender is male 

#### Import each of the 3 tables to a Pandas Dataframes

In [ ]:
import pandas as pd
employees = pd.read_sql("SELECT * FROM EmployeeBase;", conn)
recipients = pd.read_sql("SELECT * FROM RecipientBase;", conn)
messages = pd.read_sql("SELECT * FROM MessageBase;", conn)

In [ ]:
recipients.head(10)

## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [ ]:
len(employees)

In [ ]:
len(messages)

In [ ]:
# Plotting historgram with Pandas Hist
%matplotlib inline
recipients.groupby('mid')['to_eid'].agg(['count'])['count'].hist(bins=50)

In [ ]:
from collections import Counter

In [ ]:
# using the below summary you can answer Question 3
counts = Counter(recipients.groupby('mid')['to_eid'].count())
counts

In [ ]:
from bokeh.plotting import figure,show,output_notebook
output_notebook()

In [ ]:
x = [i[0] for i in counts.most_common()]
y = [i[1] for i in counts.most_common()]
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

Rescale to investigate the tail of the curve

In [ ]:
x = [i[0] for i in counts.most_common()[5:]]  # chop off the first 5
y = [i[1] for i in counts.most_common()[5:]]  # chop off the first 5
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

In [ ]:
employees.gender.value_counts()

More men

In [ ]:
employees.gender.value_counts() / employees.gender.count()

In [ ]:
# How is gender distributed across departments?
employees.groupby('department')['gender'].value_counts() / employees.groupby('department')['gender'].count()

    Forestry 83% Male
    Legal    48% Male
    Other    72% Male
    Company  72% Male

In [ ]:
# Who is sending more emails? Men or Women?
df = pd.merge(employees, messages, left_on='eid', right_on='from_eid')
df.gender.value_counts() / df.gender.count()

In [ ]:
# What's the average number of emails sent by each gender?
df.gender.value_counts() / employees.gender.value_counts()

Women sent almost twice as many messages on average

In [ ]:
employees.seniority.value_counts()

In [ ]:
df.seniority.value_counts()

In [ ]:
df.seniority.value_counts() / employees.seniority.value_counts()

Senior employees send more messages in absolute value and also on average

In [ ]:
# Which department is sending more emails? How does that relate with the number of employees in the department?
df.department.value_counts()

In [ ]:
df.department.value_counts() / employees.department.value_counts()

Legal is sending many more messages than the other departments

In [ ]:
# Who are the top 5 senders of emails? (people who sent out the most emails)
top5senders = df.eid.value_counts().head().reset_index()
top5senders.columns = ['eid', 'msgs_sent']
top5senders

In [ ]:
pd.merge(employees, top5senders, on='eid')